In [2]:
from tkinter import *
import import_ipynb
import PyPDF2
import tkinter as tk
import PIL
from PIL import ImageTk
from PIL import Image
import datetime
import sqlite3
#import Checker
from tkinter import messagebox
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="data"
)
mycursor = mydb.cursor()
image = Image.open('Unity Loss.gif')
image = image.resize((125, 100), Image.ANTIALIAS)


class Check(Toplevel):
    
    def __init__(self,t_id):
        
        Toplevel.__init__(self)
        self.geometry("650x650+500+100")
        self.title("Add Answer")
        self.resizable(False,False)
        
        #Frames
        self.Teacher_Answer=""
        self.Student_Answer=""
        self.Similarity_Count=0
        self.Similarity_Count1=0
        self.current=0
        self.top = Frame(self, height=150, bg='white')
        self.top.pack(fill=X)
        self.bottomFrame = Frame(self, height=600, bg='#57a0d3')
        self.bottomFrame.pack(fill=X)

        #Heading, image and date
        self.top_image= ImageTk.PhotoImage(image)
        self.top_image_lbl=tk.Label(self.top,image=self.top_image,bg='white')
        self.top_image_lbl.place(x=120,y=10)
        self.heading=Label(self.top,text='Check Answer',font='arial 15 bold',fg='black',bg='white')
        self.heading.place(x=330,y=60)
        
        self.lbl_course=tk.Label(self.bottomFrame,text='Select Test',font='Helvetica 9 ',fg='black',bg='#57a0d3')
        self.lbl_course.place(x=100,y=100)
        
        
        
        
        
        name=t_id
        #select `name` from `course`,`T_course` where id=%s
        print(name)
        mycursor.execute("select id,`name` from `test` ")
        OptionList = mycursor.fetchall()
        print("Printing id")
        C_id=OptionList[0]
        C_id=C_id[0]
        print(C_id)
        print("DONE")
        
        self.variable = tk.StringVar(self)
        self.variable.set(OptionList[0])
        self.opt = tk.OptionMenu(self, self.variable, *OptionList,command=self.show_answer)
        self.opt.config(width=13,height=1, font=('Helvetica', 7),bg='light blue')
        self.opt.pack()
        self.opt.place(x=225,y=250)
        r=self.variable.get()
        
        
        self.seq=int(C_id)
        
        
        mycursor.execute("select t_ans.answer,t_ans.Q_id from t_ans,test_q where t_id=%s AND t_ans.q_id=test_q.Q_id",(self.seq,))
        self.OptionList2 = mycursor.fetchall()
        print(self.OptionList2)
        print("DONE again")
        
        self.variable1 = tk.StringVar(self)
        self.variable1.set(self.OptionList2[0])
        self.opt1 = tk.OptionMenu(self, self.variable1, *self.OptionList2)
        self.opt1.config(width=13,height=1, font=('Helvetica', 7),bg='light blue')
        self.opt1.pack()
        self.opt1.place(x=225,y=350)

        button3=Button(self.bottomFrame,text='      Check Answer     ',bg="#000029",fg="white",command=lambda:self.addPerson())
        button3.place(x=480,y=470)
        self.lift()
    
    
    def addPerson(self):
        ide=self.variable.get()
        print(ide)
        ide=self.Get_S_2(ide)
        
        mycursor.execute("select t_ans.Q_id,t_ans.answer from t_ans,test_q where t_id=%s AND t_ans.q_id=test_q.Q_id",(ide,))
        self.OptionList2 = mycursor.fetchall()
        #print(self.OptionList2)
        #print("DONE again")
        
        mycursor.execute("select st_answers.Q_id,st_answers.answer from st_answers,test_q where t_id=%s AND st_answers.q_id=test_q.Q_id Order BY Q_id",(ide,))
        self.OptionList1 = mycursor.fetchall()
        #print(self.OptionList1)
        #print("DONE again")
        
        Teacher=""
        Student=""
        print("Teacher                     Student")
        self.current=self.OptionList2[0][0]
        mycursor.execute("select question from question where id=%s",(self.current,))
        List1 = mycursor.fetchall()
        strl=str(List1)
        k=len(strl)
        print("Question : "+strl[3:k-4])
        for i in range ((len(self.OptionList1))):
            
            ide=self.OptionList1[i][0]
            mycursor.execute("select t_ans.Q_id,t_ans.answer from t_ans where Q_id=%s",(ide,))
            self.OptionList2 = mycursor.fetchall()
            
            for j in range ((len(self.OptionList2))):
                
                Teacher=str(self.OptionList2[j][1])
                Student=str(self.OptionList1[i][1])
                Simi= CheckSim(Teacher,Student)
                if(Simi>self.Similarity_Count):
                    self.Similarity_Count=Simi
                    self.Similarity_Count1=Simi
                    self.Teacher_Answer=Teacher
                    self.Student_Answer=Student
                        #print(Simi)

            print()
            print("Teacher Answer : "+str(self.Teacher_Answer) +" \n\nStudent Answer : "+ str(self.Student_Answer)+ "\nSimilarity : "+ str(self.Similarity_Count) +"\n////////")
            if(ide!=self.current):
                print()
                mycursor.execute("select question from question where id=%s",(ide,))
                List1 = mycursor.fetchall()
                print("\n//////////////////////////////////////////////////////\n")
                strl=str(List1)
                k=len(strl)
                print("Question : "+strl[3:k-4])
                self.current=ide
            #print("IM HEREE                          ..........................")
            self.Similarity_Count=0
            print()

                
        print("Teacher Answer : "+str(self.Teacher_Answer) +" \n\nStudent Answer : "+ str(self.Student_Answer)+ "\nSimilarity : "+ str(self.Similarity_Count1) +"\n////////")
    
    def Get_S_2(self,ide):
        #print(ide)
        counter =0
        for i in range (len(ide)):
            if(ide[i]==','):
                break;
            else:
                counter=counter+1
        #print(counter)
        ide=ide[1:counter]
        return ide
    
    def Get_S(self,ide):
        #print(ide)
        counter =-1
        for i in range (len(ide)):
            if(ide[i]==','):
                break;
            else:
                counter=counter+1
        #print(counter)
        ide=ide[2:counter]
        return ide
    
    def show_answer(self,var):
        #print("Working")
        
        ide=self.variable.get()
        print(ide)
        ide=self.Get_S_2(ide)
        #print(ide)
        
        mycursor.execute("select t_ans.answer,t_ans.Q_id from t_ans,test_q where t_id=%s AND t_ans.q_id=test_q.Q_id",(ide,))
        self.OptionList2 = mycursor.fetchall()
        print(self.OptionList2)
        
        if not self.OptionList2:
            #print(self.OptionList2)
            self.OptionList2=["None"]
            self.variable1.set(self.OptionList2)
            self.opt1 = tk.OptionMenu(self, self.variable1, *self.OptionList2)
            self.opt1.config(width=13,height=1, font=('Helvetica', 7),bg='light blue')
            self.opt1.pack()
            self.opt1.place(x=225,y=350)
        else:
            self.variable1.set(self.OptionList2[0])
            self.opt1 = tk.OptionMenu(self, self.variable1, *self.OptionList2)
            self.opt1.config(width=13,height=1, font=('Helvetica', 7),bg='light blue')
            self.opt1.pack()
            self.opt1.place(x=225,y=350)
        


admin
Printing id
1
DONE
[('Refactoring is the process of changing a software system in such a way that it does not alter the function of the code yet improves its internal structure.', 12), ('Refactoring consists of improving the internal structure of an existing program’s source code, while preserving its external behavior.', 12), ('It is defined as systematic process of improving existing computer code, without adding new functionality or changing external behaviour of the code.', 12), (' is software testing technique in which internal structure, design and coding of software are tested to verify flow of input-output and to improve design', 13), ('also known as clear box testing, glass box testing, transparent box testing, and structural testing) is a method of software testing that tests internal structures or workings of an application, as opposed to its functionality (i.e. black-box testing). In white-box testing an internal perspective of the system, as well as programming skill

In [ ]:
#import import_ipynb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import concurrent.futures

batch_size = 32
max_length = 512 

# Labels in our dataset.
labels = ["contradiction", "entailment", "neutral"]


class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array as input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)
            
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#strategy = tf.distribute.MirroredStrategy()

def create_model():
    #with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False
    
    sequence_output, pooled_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    #print("sequence_output")
    #print(sequence_output)
    #print("sequence_output")
    #print(pooled_output)

    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(256, return_sequences=True)
    )(sequence_output)
    #print("Bi-Lstm.......")
    #print(bi_lstm)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    #print("avg_pool.......")
    #print(avg_pool)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    #print("max_pool.......")
    #print(max_pool)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    #print("concat.......")
    #print(concat)
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    #print("dropout.......")
    #print(dropout)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    #print("output.......")
    #print(output)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )
    return model,bert_model

#print(f"Strategy: {strategy}")
model,bert_model = create_model()
#model.summary()

l = tf.train.latest_checkpoint("model/")
model.load_weights(l)

def check_(sentence_pairs):
    print("Check .... calling")
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    print("1")
    proba = model.predict(test_data)[0]
    print("2")
    idx = np.argmax(proba)
    print("3")
    proba = f"{proba[idx]: .2f}%"
    print("4")
    pred = labels[idx]
    print("Checked returned")
    return pred, proba

def CheckSim(Teacher,Student):
    #print("Question coming_1")
    ## if Student Answer goes greater than 512 so apply this 
    s= []
    
    while len(Student)>512:
        stop=1
        for i in range(500,512):
            if stop and (Student[i]=='.' or Student[i]==',' or Student[i]==' ' or Student[i]==':'): 
                s.append([Student[:i+1]])
                Student = Student[i+1:]
                #print(len(Student))
                stop=0
    s.append(Student)
    
    #print("Student answer divided")
    
    sentence_pairs = []
    
    for i in s:
        sentence_pairs.append(np.array([[str(Teacher), str(i)]]))

    result = []
    grade =  []

    #print("thread phase")
    #results = 0
    #with concurrent.futures.ThreadPoolExecutor() as executor:
    #    results= executor.map(check_, sentence_pairs)
    for i in sentence_pairs:
        #print("Check .... calling")
        test_data = BertSemanticDataGenerator(
            i, labels=None, batch_size=1, shuffle=False, include_targets=False,
        )
        #print("1")
        proba = model.predict(test_data)[0]
        #print("2")
        idx = np.argmax(proba)
        #print("3")
        proba = f"{proba[idx]: .2f}%"
        #print("4")
        pred = labels[idx]
        #print("Checked returned")
        grade.append(pred)
        result.append(proba)
        #return pred, proba

    #print("Result ... calling")
    #for i in results:
    #    print("problem.....")
    #    grade.append(i[0])
    #    result.append(i[1])
    
    #print("Threads done")
    marks = []
    for i in range(0,len(grade)):
        #print(grade[i])
        #print(result[i])
        if result[i][1:5] == '1.00':
            result[i] = 100
        else:
            result[i] = int(result[i][3:5])
        #print("############")
        #print(grade[i])
        #print(result[i])
        if "contradiction" == grade[i]:
            if result[i]>=60:
                ans=result[i]-60
                ans= 40 - ans
                marks.append(ans)
        if "neutral" == grade[i]:
            if result[i]>=60:
                ans=result[i]-60
                ans=(ans/2)+60
                marks.append(ans)
        if "entailment" == grade[i]:
            if result[i]>=60:
                ans=result[i]-60
                ans=(ans/2)+80
                marks.append(ans)
    
    if not marks:
        return 0
    n =len(marks)
    m=0
    print("Printing Marks by threads : {}".format(marks))
    for i in range(0,n):
        m = m+marks[i]
    return m/n
    
#sentence1 = "It should be re-engineered or replaced if suitable system is available, this is so because of its high business value it contributes a lot to the business."
#sentence2 = "High Quality with High Business value as the system is somehow important to the business and as the change cannot be avoided further in this system and is running for long time."

#CheckSim(sentence1, sentence2)

In [ ]:
def main():
    root = Tk()
    app = Check("admin")
    root.title("Unity Loss")
    root.geometry("650x650+500+100")
    root.resizable(False,False)
    root.mainloop()
if __name__ == '__main__':
    main()